# IFC Rasterization for Efficient Spatial Analysis

## Intro

### Import libraries and the IFC model:

In [6]:
import pyvista as pv
import numpy as np
from typing import Any, Tuple
import ifcopenshell
import logging
from importlib import reload 
import multiprocessing
import ifcopenshell
import ifcopenshell.geom
import time
import vtk
#ifc_file = ifcopenshell.open(r"IFC Files\Project1.ifc")
ifc_file = ifcopenshell.open(r"IFC Files\Duplex.ifc")

### Logging setup:

In [7]:
#set up logging
reload(logging)
logging.basicConfig(format='%(asctime)s %(levelname)s:%(message)s', level=logging.INFO, filename="test.log",  datefmt='%I:%M:%S')

# Configure logging to write to a file
# logging.basicConfig(filename='./app.log',  level=logging.INFO)
logname="./vista.log"
logging.basicConfig(filename=logname,
                    filemode='a',
                    format='%(asctime)s,%(msecs)d %(name)s %(levelname)s %(message)s',
                    datefmt='%H:%M:%S',
                    level=logging.INFO)

logging.info("Running IFC Voxelizer")

logger = logging.getLogger('IFCVoxelizer')
logger.debug("hello")

## Functions

In [8]:
def to_vtk_faces(faces : Tuple[tuple]) -> np.ndarray:
    faces=np.array(faces, dtype=np.int16)
    num_insertions = (len(faces) - 1) // 3
    # Generate an array of indices for insertions
    indices = np.arange(3, 3 * (num_insertions + 1), 3)
    indices = np.insert(indices, 0, 0)
    faces = np.insert(faces, indices, 3)
    return faces

def vtk_block_by_building_element(ifc_file):
    building_elements = ifc_file.by_type("IfcBuildingElement")
    settings = ifcopenshell.geom.settings()
    settings.set(settings.USE_WORLD_COORDS, True)
    settings.set(settings.APPLY_DEFAULT_MATERIALS, True)
    iterator = ifcopenshell.geom.iterator(settings, ifc_file, multiprocessing.cpu_count())
    multiblock = pv.MultiBlock()
    element_information = {} # Dictionary to hold element information
    exclude_list = ["IfcSpace", "IfcOpeningElement"]

    if iterator.initialize():
        while True:
            shape = iterator.get()
            if shape.type not in exclude_list:
                element = ifc_file.by_guid(shape.guid)
                           
                faces = shape.geometry.faces
                verts = shape.geometry.verts
                poly_data = pv.PolyData(list(verts), to_vtk_faces(faces))
                multiblock.append(poly_data)
                  
                if element in building_elements:
                #print(element.all_attributes()) --> why doesn't it work?
                        
                    element_information[shape.guid] = {
                    "Geo": poly_data, 
                    "Type": shape.type,
                    "Name": element.Name,
                    "Description": element.Description
                    }
                               
            if not iterator.next():
                break
                
    return multiblock, element_information

def create_uniform_grid(bounds, voxel_size):
    """Create a uniform grid within the given bounds."""
    x = np.arange(bounds[0], bounds[1] + voxel_size, voxel_size)
    y = np.arange(bounds[2], bounds[3] + voxel_size, voxel_size)
    z = np.arange(bounds[4], bounds[5] + voxel_size, voxel_size)
    return pv.StructuredGrid(*np.meshgrid(x, y, z))

def boxes_touch(A, B):
    """
    Check if two 3D bounding boxes touch or overlap.
    
    Parameters:
    - A, B: Tuples representing the bounds of boxes A and B.
      Each tuple should be in the format (xmin, xmax, ymin, ymax, zmin, zmax).
    
    Returns:
    - True if the boxes touch or overlap, otherwise False.
    """
    
    # Check for overlap in the x, y, and z dimensions
    overlap_x = A[0] <= B[1] and A[1] >= B[0]
    overlap_y = A[2] <= B[3] and A[3] >= B[2]
    overlap_z = A[4] <= B[5] and A[5] >= B[4]
    
    # Return True if all dimensions overlap, otherwise False
    return overlap_x and overlap_y and overlap_z


In [9]:
def voxelize_space(meshes, voxel_size, mesh_info):
    """Voxelize space and check intersections with given mesh."""
    grid = create_uniform_grid(meshes.bounds, voxel_size)

    num_points = grid.cell_centers().n_points

    mask2 = np.zeros(num_points, dtype=bool)

    cell_to_guid = {}

    print(f'Total number of voxels: {num_points}')

    for guid, mesh_data in mesh_info.items():

        mesh = mesh_data['Geo']

        mesh_bounds = mesh.bounds
        
        for i in range(num_points):

            cell = grid.extract_cells(i)
            cell_bounds = cell.bounds         

            if boxes_touch(mesh_bounds, cell_bounds):
                if not mask2[i]:
                    mask2[i] = True
                    cell_to_guid[i] = guid          
                            
    return grid, mask2, cell_to_guid

## Voxelization

In [10]:
p = pv.Plotter()
print(time.time())

all_meshes, info = vtk_block_by_building_element(ifc_file)

print(time.time())


start_time = time.time()

voxel_size = 0.5

# Voxelize the entire space of the combined mesh
grid, mask2, cell_info = voxelize_space(all_meshes, voxel_size, info)

end_time = time.time()

'''# Prepare to store cell-IFC-object mappings
cell_data = {}

for cell_id, guid in cell_info.items():
    cell_data[cell_id] = info[guid]
'''

elapsed_time = end_time - start_time

print(f"Time taken to voxelize: {elapsed_time:.4f} seconds")

# Visualization
p.add_mesh(grid, opacity=0.3, show_edges=True)

p.add_mesh(grid.extract_cells(np.where(mask2)[0]), color="red", opacity=0.5)

p.show()





1694173266.3525405
1694173269.6113575
Total number of voxels: 17442


KeyboardInterrupt: 